In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
def read_csv(policy_name, dir, is_SB, case):
    dataframes = []
    filenames = []

    for root, dirs, files in os.walk(dir):
        # Filter directories within the root directory
        dirs[:] = [d for d in dirs if d.startswith(policy_name)]
        # Only proceed if we're in a directory that matches the folder prefix
        if os.path.basename(root).startswith(policy_name):
            # Loop through each file in the directory
            for file in files:
                # Check if file starts with the prefix
                if file.startswith(case):
                    file_path = os.path.join(root, file)
                    try:
                        # Load the file into a DataFrame
                        df = pd.read_csv(file_path)
                        # Store the DataFrame in the dictionary using the path as a key
                        dataframes.append(df)
                        filenames.append(file)
                    except Exception as e:
                        print(f"Failed to read {file_path}: {e}")
    columns_sb = ['Bike Arrivals', 'Bike Starvations', 'Long Congestions']
    
    columns_ff = ['Escooter Arrivals', 'Escooter Starvations', 'Battery Starvations', 'Battery Violations']
    columns_sol_time = ['Accumulated solution time', 'Number of get_best_action']
    
    fix_dfs = []
    for df in dataframes:
        if case == 'sol_time':
            selected_columns = columns_sol_time 
        elif is_SB:
            selected_columns = columns_sb
        else:
            selected_columns = columns_ff
        
        fix_dfs.append(df[selected_columns])
    return fix_dfs, filenames

In [3]:
def make_pie(column_means):
    # Calculate the percentage contribution of each column's mean
    total_means = column_means.sum()
    percentage_means = (column_means / total_means) * 100

    # Plotting the pie chart
    plt.figure(figsize=(8, 8))  # Adjust the figure size as necessary
    plt.pie(percentage_means, labels=percentage_means.index, autopct='%1.1f%%', startangle=90)
    
    plt.show()

### Rank the results

In [12]:
dir = '/Users/isabellam/NTNU/H2023/Prosjektoppgave/fomo/policies/hlv_master/all_results/discount_results/'
dataframes, filenames = read_csv('Base1', dir, False, 'discount')
filenames

['discount_factor_0.4_Base.csv',
 'discount_factor_0.5_Base.csv',
 'discount_factor_0.9_Base.csv',
 'discount_factor_0.8_Base.csv',
 'discount_factor_1.0_Base.csv',
 'discount_factor_0.3_Base.csv',
 'discount_factor_0.2_Base.csv',
 'discount_factor_0.7_Base.csv',
 'discount_factor_0.6_Base.csv',
 'discount_factor_0.1_Base.csv']

In [13]:
means = []
for i in range(len(dataframes)):
    dataframe = dataframes[i]
    filename = filenames[i]
    
    mean_series = dataframe.mean()
    mean_series['Filename'] = filename
    means.append(mean_series)

new_df = pd.DataFrame(means)
new_df['Sum'] = new_df.drop(columns=['Filename', 'Escooter Arrivals']).sum(axis=1)
new_df['Lost trips percentage'] = round(new_df['Sum'] / new_df[['Escooter Arrivals', 'Sum']].sum(axis=1) * 100, 2)
new_df['Rank failed events'] = new_df['Sum'].rank(method='min')
new_df['Rank p'] = new_df['Lost trips percentage'].rank(method='min')

new_column_order = ['Filename', 'Escooter Arrivals', 'Escooter Starvations', 'Battery Starvations', 'Battery Violations', 'Sum', 'Lost trips percentage', 'Rank failed events', 'Rank p']
new_df = new_df[new_column_order]
new_df = new_df.sort_values('Filename')

new_df

,Filename,Escooter Arrivals,Escooter Starvations,Battery Starvations,Battery Violations,Sum,Lost trips percentage,Rank failed events,Rank p
9,discount_factor_0.1_Base.csv,26195.263158,13136.736842,1323.684211,0.0,14460.421053,35.57,6.0,6.0
6,discount_factor_0.2_Base.csv,26165.210526,13244.894737,1255.473684,0.0,14500.368421,35.66,10.0,10.0
5,discount_factor_0.3_Base.csv,26225.631579,13148.684211,1276.842105,0.0,14425.526316,35.49,4.0,4.0
0,discount_factor_0.4_Base.csv,26248.000000,13125.526316,1323.000000,0.0,14448.526316,35.50,5.0,5.0
1,discount_factor_0.5_Base.csv,26171.526316,13182.315789,1296.421053,0.0,14478.736842,35.62,8.0,9.0
8,discount_factor_0.6_Base.csv,26199.000000,13169.315789,1320.000000,0.0,14489.315789,35.61,9.0,8.0
7,discount_factor_0.7_Base.csv,26214.473684,13208.684211,1269.631579,0.0,14478.315789,35.58,7.0,7.0
3,discount_factor_0.8_Base.csv,26257.315789,13120.105263,1288.578947,0.0,14408.684211,35.43,2.0,2.0
2,discount_factor_0.9_Base.csv,26266.473684,13101.894737,1321.210526,0.0,14423.105263,35.45,3.0,3.0
4,discount_factor_1.0_Base.csv,26262.894737,13134.052632,1243.473684,0.0,14377.526316,35.38,1.0,1.0


## SB TABLE

In [12]:
dataframes2, filenames2 = read_csv('Collab3', dir, True, 'adjustment')

means2 = []
for i in range(len(dataframes2)):
    dataframe2 = dataframes2[i]
    filename2 = filenames2[i]
    
    mean_series2 = dataframe2.mean()
    mean_series2['Filename'] = filename2
    means2.append(mean_series2)

new_df2 = pd.DataFrame(means2)
new_df2['Sum'] = new_df2.drop(columns=['Filename', 'Bike Arrivals']).sum(axis=1)
new_df2['Lost trips percentage'] = round(new_df2['Sum'] / new_df2[['Bike Arrivals', 'Sum']].sum(axis=1) * 100, 2)
new_df2['Rank failed events'] = new_df2['Sum'].rank(method='min')
new_df2['Rank p'] = new_df2['Lost trips percentage'].rank(method='min')

new_column_order2 = ['Filename', 'Bike Arrivals', 'Bike Starvations', 'Long Congestions', 'Sum', 'Lost trips percentage', 'Rank failed events', 'Rank p']
new_df2 = new_df2[new_column_order2]
new_df2 = new_df2.sort_values('Filename')

new_df2

KeyError: "['Filename', 'Bike Arrivals'] not found in axis"

In [ ]:
merged_df = pd.merge(new_df, new_df2, on='Filename', how='inner')
df = merged_df
df['Rank sum'] = df['Rank failed events_x'] +df['Rank failed events_y']
df

,Filename,Escooter Arrivals,Escooter Starvations,Battery Starvations,Battery Violations,Sum_x,Lost trips percentage_x,Rank failed events_x,Rank p_x,Bike Arrivals,Bike Starvations,Long Congestions,Sum_y,Lost trips percentage_y,Rank failed events_y,Rank p_y,Rank sum
0,adjustment_factor_0.25_Collab3.csv,12571.210526,17742.473684,10360.947368,0.0,28103.421053,69.09,17.0,17.0,11729.894737,130.263158,1064.526316,1194.789474,9.24,1.0,1.0,18.0
1,adjustment_factor_0.5_Collab3.csv,13459.894737,17461.526316,9701.789474,0.0,27163.315789,66.87,16.0,16.0,11722.736842,124.578947,1148.473684,1273.052632,9.80,2.0,2.0,18.0
2,adjustment_factor_0.6_Collab3.csv,14035.421053,17465.578947,9174.263158,0.0,26639.842105,65.49,15.0,15.0,11808.842105,107.000000,1192.894737,1299.894737,9.92,4.0,3.0,19.0
3,adjustment_factor_0.7_Collab3.csv,14134.157895,17389.315789,9139.842105,0.0,26529.157895,65.24,14.0,14.0,11786.842105,110.526316,1224.105263,1334.631579,10.17,7.0,5.0,21.0
4,adjustment_factor_0.8_Collab3.csv,14771.789474,17329.368421,8612.526316,0.0,25941.894737,63.72,13.0,13.0,11719.631579,102.105263,1191.684211,1293.789474,9.94,3.0,4.0,16.0
5,adjustment_factor_0.9_Collab3.csv,14767.526316,17336.842105,8534.578947,0.0,25871.421053,63.66,12.0,12.0,11775.052632,105.736842,1228.421053,1334.157895,10.18,6.0,6.0,18.0
6,adjustment_factor_1.1_Collab3.csv,16124.000000,16969.736842,7561.947368,0.0,24531.684211,60.34,10.0,10.0,11726.736842,109.684211,1430.421053,1540.105263,11.61,8.0,8.0,18.0
7,adjustment_factor_1.2_Collab3.csv,16376.684211,16837.052632,7494.052632,0.0,24331.105263,59.77,9.0,9.0,11753.947368,97.526316,1445.631579,1543.157895,11.61,9.0,8.0,18.0
8,adjustment_factor_1.3_Collab3.csv,16864.315789,16772.315789,7027.000000,0.0,23799.315789,58.53,8.0,8.0,11773.526316,109.368421,1503.000000,1612.368421,12.05,10.0,10.0,18.0
9,adjustment_factor_1.4_Collab3.csv,17470.421053,16509.421053,6643.473684,0.0,23152.894737,56.99,7.0,7.0,11766.578947,120.210526,1606.789474,1727.000000,12.80,13.0,12.0,20.0


In [18]:
df['Final Rank'] = df[['Rank p_x', 'Rank p_y']].max(axis=1)
df

,Filename,Escooter Arrivals,Escooter Starvations,Battery Starvations,Battery Violations,Sum_x,Lost trips percentage_x,Rank failed events_x,Rank p_x,Bike Arrivals,Bike Starvations,Long Congestions,Sum_y,Lost trips percentage_y,Rank failed events_y,Rank p_y,Rank sum,Final Rank
0,adjustment_factor_0.25_Collab3.csv,12571.210526,17742.473684,10360.947368,0.0,28103.421053,69.09,17.0,17.0,11729.894737,130.263158,1064.526316,1194.789474,9.24,1.0,1.0,18.0,17.0
1,adjustment_factor_0.5_Collab3.csv,13459.894737,17461.526316,9701.789474,0.0,27163.315789,66.87,16.0,16.0,11722.736842,124.578947,1148.473684,1273.052632,9.80,2.0,2.0,18.0,16.0
2,adjustment_factor_0.6_Collab3.csv,14035.421053,17465.578947,9174.263158,0.0,26639.842105,65.49,15.0,15.0,11808.842105,107.000000,1192.894737,1299.894737,9.92,4.0,3.0,19.0,15.0
3,adjustment_factor_0.7_Collab3.csv,14134.157895,17389.315789,9139.842105,0.0,26529.157895,65.24,14.0,14.0,11786.842105,110.526316,1224.105263,1334.631579,10.17,7.0,5.0,21.0,14.0
4,adjustment_factor_0.8_Collab3.csv,14771.789474,17329.368421,8612.526316,0.0,25941.894737,63.72,13.0,13.0,11719.631579,102.105263,1191.684211,1293.789474,9.94,3.0,4.0,16.0,13.0
5,adjustment_factor_0.9_Collab3.csv,14767.526316,17336.842105,8534.578947,0.0,25871.421053,63.66,12.0,12.0,11775.052632,105.736842,1228.421053,1334.157895,10.18,6.0,6.0,18.0,12.0
6,adjustment_factor_1.1_Collab3.csv,16124.000000,16969.736842,7561.947368,0.0,24531.684211,60.34,10.0,10.0,11726.736842,109.684211,1430.421053,1540.105263,11.61,8.0,8.0,18.0,10.0
7,adjustment_factor_1.2_Collab3.csv,16376.684211,16837.052632,7494.052632,0.0,24331.105263,59.77,9.0,9.0,11753.947368,97.526316,1445.631579,1543.157895,11.61,9.0,8.0,18.0,9.0
8,adjustment_factor_1.3_Collab3.csv,16864.315789,16772.315789,7027.000000,0.0,23799.315789,58.53,8.0,8.0,11773.526316,109.368421,1503.000000,1612.368421,12.05,10.0,10.0,18.0,10.0
9,adjustment_factor_1.4_Collab3.csv,17470.421053,16509.421053,6643.473684,0.0,23152.894737,56.99,7.0,7.0,11766.578947,120.210526,1606.789474,1727.000000,12.80,13.0,12.0,20.0,12.0


## Merge with solution time

In [73]:
dataframes2, filenames2 = read_csv('Base', dir, True, 'sol_time')
filenames2

['sol_time_num_scenarios_120_Base.csv',
 'sol_time_num_scenarios_20_Base.csv',
 'sol_time_num_scenarios_90_Base.csv',
 'sol_time_num_scenarios_40_Base.csv',
 'sol_time_num_scenarios_30_Base.csv',
 'sol_time_num_scenarios_80_Base.csv',
 'sol_time_num_scenarios_50_Base.csv',
 'sol_time_num_scenarios_100_Base.csv',
 'sol_time_num_scenarios_60_Base.csv',
 'sol_time_num_scenarios_1_Base.csv',
 'sol_time_num_scenarios_10_Base.csv',
 'sol_time_num_scenarios_70_Base.csv']

In [74]:
import numpy as np

means2 = []
for i in range(len(dataframes2)):
    dataframe2 = dataframes2[i]
    filename2 = filenames2[i]
    timehor = filename2.split('_')[4]
    
    mean_series2 = dataframe2.mean()
    mean_series2['Filename'] = filename2
    mean_series2['Time Horizon'] = int(timehor)
    means2.append(mean_series2)

new_df2 = pd.DataFrame(means2)

new_df2['avg_solution_time'] = new_df2['Accumulated solution time'].astype(float) / new_df2['Number of get_best_action'].replace(0, np.nan).astype(float)

new_df2['Rank sol_time'] = new_df2['avg_solution_time'].rank(method='min')
new_df2['Filename'] = new_df2['Filename'].str[9:]

new_column_order1 = ['Filename', 'Time Horizon', 'Accumulated solution time', 'Number of get_best_action', 'avg_solution_time', 'Rank sol_time']
new_df2 = new_df2[new_column_order1]
new_df2 = new_df2.sort_values('Filename')

new_df2

,Filename,Time Horizon,Accumulated solution time,Number of get_best_action,avg_solution_time,Rank sol_time
7,num_scenarios_100_Base.csv,100,13589.097505,2268.894737,5.989303,10.0
10,num_scenarios_10_Base.csv,10,4095.839549,2163.789474,1.892901,2.0
0,num_scenarios_120_Base.csv,120,15925.570435,2248.263158,7.083499,12.0
9,num_scenarios_1_Base.csv,1,3099.116804,2408.052632,1.286981,1.0
1,num_scenarios_20_Base.csv,20,5123.110717,2220.684211,2.306997,3.0
4,num_scenarios_30_Base.csv,30,6121.158552,2300.736842,2.660521,4.0
3,num_scenarios_40_Base.csv,40,7087.922840,2329.631579,3.042508,5.0
6,num_scenarios_50_Base.csv,50,9575.303924,2249.947368,4.255790,6.0
8,num_scenarios_60_Base.csv,60,11244.486720,2200.368421,5.110275,8.0
11,num_scenarios_70_Base.csv,70,14428.866856,2243.526316,6.431334,11.0


In [75]:
merged_df = pd.merge(new_df, new_df2, on='Filename', how='inner')
df = merged_df
df

,Filename,Escooter Arrivals,Escooter Starvations,Battery Starvations,Battery Violations,Sum,Lost trips percentage,Rank failed events,Rank p,Time Horizon,Accumulated solution time,Number of get_best_action,avg_solution_time,Rank sol_time
0,num_scenarios_100_Base.csv,26610.684211,12883.789474,1156.736842,0.0,14040.526316,34.54,3.0,3.0,100,13589.097505,2268.894737,5.989303,10.0
1,num_scenarios_10_Base.csv,26496.894737,13014.631579,1109.368421,0.0,14124.000000,34.77,8.0,10.0,10,4095.839549,2163.789474,1.892901,2.0
2,num_scenarios_120_Base.csv,26475.789474,12944.368421,1187.526316,0.0,14131.894737,34.80,10.0,12.0,120,15925.570435,2248.263158,7.083499,12.0
3,num_scenarios_1_Base.csv,26701.000000,12799.473684,1134.000000,0.0,13933.473684,34.29,1.0,1.0,1,3099.116804,2408.052632,1.286981,1.0
4,num_scenarios_20_Base.csv,26622.473684,12985.631579,1139.947368,0.0,14125.578947,34.67,9.0,6.0,20,5123.110717,2220.684211,2.306997,3.0
5,num_scenarios_30_Base.csv,26606.368421,12913.000000,1177.315789,0.0,14090.315789,34.62,5.0,5.0,30,6121.158552,2300.736842,2.660521,4.0
6,num_scenarios_40_Base.csv,26536.578947,13012.842105,1144.000000,0.0,14156.842105,34.79,12.0,11.0,40,7087.922840,2329.631579,3.042508,5.0
7,num_scenarios_50_Base.csv,26606.263158,12916.000000,1144.105263,0.0,14060.105263,34.57,4.0,4.0,50,9575.303924,2249.947368,4.255790,6.0
8,num_scenarios_60_Base.csv,26572.105263,13006.263158,1099.473684,0.0,14105.736842,34.68,6.0,7.0,60,11244.486720,2200.368421,5.110275,8.0
9,num_scenarios_70_Base.csv,26541.947368,13017.315789,1122.157895,0.0,14139.473684,34.76,11.0,9.0,70,14428.866856,2243.526316,6.431334,11.0


In [9]:
import matplotlib.ticker as ticker

# Assuming 'df' is your DataFrame
# Pivot your DataFrame as needed (if it is not already in the right format)
df1 = df[df['time'][2].astype(int)]
pivoted_df = df1.pivot(index='b', columns='a', values='Sum')

colors = ['#B4D3BA', '#86A86B', '#B57F50', '#4B543B', '#DB6F10', '#00B307', '#FFB674']

# Start plotting
fig, ax = plt.subplots(figsize=(10, 6))  # You can adjust the size as needed

# Plot each column

for i, column in enumerate(pivoted_df.columns):
    color = colors[i % len(colors)]  # Cycle through the color list
    ax.plot(pivoted_df.index, pivoted_df[column], marker='o', color=color, label=f'α = {column}')

# Customize the plot to match the uploaded image
ax.set_title('Number of failed events for various combinations of α and β')
ax.set_xlabel('β')
ax.set_ylabel('Failed events')
ax.legend(loc='upper left')

# It looks like the original plot uses a specific tick locator for the x-axis
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Show the plot
plt.show()

KeyError: 'b'

In [15]:
import matplotlib.ticker as ticker

# Assuming 'df' is your DataFrame
# Pivot your DataFrame as needed (if it is not already in the right format)
df1 = df[df['b'].astype(int) < 6]
pivoted_df = df1.pivot(index='b', columns='a', values='avg_solution_time')

colors = ['#B4D3BA', '#86A86B', '#B57F50', '#4B543B', '#DB6F10', '#00B307', '#FFB674']

# Start plotting
fig, ax = plt.subplots(figsize=(10, 6))  # You can adjust the size as needed

# Plot each column

for i, column in enumerate(pivoted_df.columns):
    color = colors[i % len(colors)]  # Cycle through the color list
    ax.plot(pivoted_df.index, pivoted_df[column], marker='o', color=color, label=f'α = {column}')

# Customize the plot to match the uploaded image
ax.set_title('Average solution time for each subproblem for various combinations of α and β')
ax.set_xlabel('β')
ax.set_ylabel('Average Solution time')
ax.legend(loc='upper left')

# It looks like the original plot uses a specific tick locator for the x-axis
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Show the plot
plt.show()

NameError: name 'df' is not defined